In [ ]:
# crypto_bot.py
# Requiere: pip install ccxt ta-lib pandas requests

import os, json, time, logging, requests
import pandas as pd
import ccxt, talib
from datetime import datetime, timezone

class CryptoBot:
    TIMEFRAME_SECONDS = {"1m": 60, "5m": 300, "15m": 900}

    def __init__(self, symbol="BTC/USDT", timeframe="1m", telegram_token="", telegram_chat_id="", tp_pct=1.5, sl_pct=0.75, strategy_name="rsi_bollinger"):
        self.symbol = symbol
        self.timeframe = timeframe
        self.tp_pct = tp_pct / 100
        self.sl_pct = sl_pct / 100
        self.strategy_name = strategy_name
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.exchange = ccxt.binance()
        self.current_trade = None
        self.timeframe_seconds = self.TIMEFRAME_SECONDS.get(timeframe, 60)
        self.strategies = {"rsi_bollinger": self.rsi_bollinger}
        self.df = pd.DataFrame()

    def run(self):
        print("🟢 Bot initialized")
        while True:
            self.fetch_data()
            self.trade()
            self.sleep_until_next_candle()

    def sleep_until_next_candle(self):
        now = int(datetime.now(timezone.utc).timestamp())
        wait = self.timeframe_seconds - (now % self.timeframe_seconds)
        time.sleep(wait)

    def fetch_data(self):
        try:
            ohlcv = self.exchange.fetch_ohlcv(self.symbol, self.timeframe, limit=20)
            df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
            self.df = df
        except Exception as e:
            self.notify(f"❌ Fetch error: {e}")
            self.df = pd.DataFrame()

    def trade(self):
        if self.strategy_name not in self.strategies or self.df.empty:
            return

        signal, reason = self.strategies[self.strategy_name]()
        if len(self.df) < 2:
            return

        price = self.df.iloc[-1]["close"]

        if self.current_trade is None and signal in ["long", "short"]:
            self.enter(signal, price, reason)
        elif self.current_trade:
            direction = self.current_trade["direction"]
            entry = self.current_trade["entry_price"]
            tp_hit = price >= entry * (1 + self.tp_pct) if direction == "long" else price <= entry * (1 - self.tp_pct)
            sl_hit = price <= entry * (1 - self.sl_pct) if direction == "long" else price >= entry * (1 + self.sl_pct)

            if tp_hit:
                self.exit(price, "TP hit")
            elif sl_hit:
                self.exit(price, "SL hit")
            elif signal == "exit":
                self.exit(price, reason)

    def enter(self, direction, price, reason):
        self.current_trade = {
            "direction": direction,
            "entry_price": price,
            "entry_time": datetime.now().isoformat()
        }
        self.notify(f"✅ Enter {direction.upper()} @ {price}\nReason: {reason}")

    def exit(self, price, reason):
        entry = self.current_trade
        profit = price - entry["entry_price"] if entry["direction"] == "long" else entry["entry_price"] - price
        pnl_pct = (profit / entry["entry_price"]) * 100
        self.notify(f"❌ Exit {entry['direction'].upper()} — {reason}\nEntry: {entry['entry_price']} → Exit: {price}\nPnL: {profit:.2f} USD ({pnl_pct:.2f}%)")
        self.current_trade = None

    def notify(self, text):
        try:
            print(text)
            requests.post(f"https://api.telegram.org/bot{self.telegram_token}/sendMessage", data={"chat_id": self.telegram_chat_id, "text": text})
        except Exception as e:
            logging.error(f"[Telegram] {e}")

    def rsi_bollinger(self):
        self.df["rsi_6"] = talib.RSI(self.df["close"], 6)
        self.df["rsi_12"] = talib.RSI(self.df["close"], 12)
        self.df["bb_upper"], _, self.df["bb_lower"] = talib.BBANDS(self.df["close"], timeperiod=20)

        last, prev = self.df.iloc[-1], self.df.iloc[-2]
        rsi6, rsi12, price = last["rsi_6"], last["rsi_12"], last["close"]
        prev_rsi6, prev_rsi12 = prev["rsi_6"], prev["rsi_12"]
        bb_upper, bb_lower = last["bb_upper"], last["bb_lower"]

        self.notify(f"\n📊 Strategy Indicators [{last['timestamp']}]")
        self.notify(f"💰 Price: {price:.2f}")
        self.notify(f"📈 RSI(6): {rsi6:.2f} | RSI(12): {rsi12:.2f}")
        self.notify(f"📊 BB Upper: {bb_upper:.2f} | BB Lower: {bb_lower:.2f}")
        self.notify(f"🔄 RSI Cross: RSI6 was {prev_rsi6:.2f}, now {rsi6:.2f} | RSI12 was {prev_rsi12:.2f}, now {rsi12:.2f}")

        if self.current_trade is None:
            if rsi6 > 70 and rsi12 > 70 and price > bb_upper:
                return "short", "RSIs > 70 and price > BB upper"
            if rsi6 < 30 and rsi12 < 30 and price < bb_lower:
                return "long", "RSIs < 30 and price < BB lower"
            return None, "No entry"
        
        direction = self.current_trade["direction"]
        cross_exit = (direction == "long" and prev_rsi6 > prev_rsi12 and rsi6 < rsi12) or \
                     (direction == "short" and prev_rsi6 < prev_rsi12 and rsi6 > rsi12)

        if cross_exit:
            return "exit", "RSI6 crossed RSI12"
        else:
            return None, "Hold"

In [ ]:
bot = CryptoBot(
    symbol="BTC/USDT",
    timeframe="5m",
    telegram_token="",
    telegram_chat_id="",
    strategy_name="rsi_bollinger"
)
bot.run()